In [ ]:
import pandas as pd
from database.connect import getConnection
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import seaborn as sns
sns.set()

In [ ]:
start_date = '01-01-2000'
end_date = '01-01-2023'

# convert start and end date to unix timestamp in milliseconds
start_date_unix = int(datetime.strptime(start_date, '%d-%m-%Y').timestamp() * 1000)
end_date_unix = int(datetime.strptime(end_date, '%d-%m-%Y').timestamp() * 1000)

# create a part you can insert into your where clause
date_restriction = f" AND tweets.timestamp_ms >= {start_date_unix} AND tweets.timestamp_ms <= {end_date_unix}"

In [ ]:
try:
    connection = getConnection()
except Exception:
    print("✖️ Error while connecting to MySQL engine database.")
    print("ℹ️ Please make sure the environment file `.env` is located at"+
        "the project root directory and contains proper configuration.")
    raise

cursor = connection.cursor()

In [ ]:
cursor.execute(f""" SELECT bs.bin_id, bs.cID, bs.bin_position, bs.break_airline, bs.sentiment_sum, bs.tweet_count, c.Tstart, c.Tend
                    FROM binned_sentiment bs, conversations c
                    WHERE bs.cID = c.id AND bs.break_airline NOT LIKE 'prev=%'""")
binned_sentiment = cursor.fetchall()
#all bins from conversations with more than 1 bin.

Ideas:
df-columns: airline, sentiment avg before and after tweet, timestamp, 

In [ ]:
query = """ SELECT CASE WHEN bs1.break_airline='AmericanAir' THEN "AmericanAir" 
                        WHEN bs1.break_airline = 'British_Airways' THEN "British Airways" 
                        ELSE "Other" END AS Airline, 
            (bs1.sentiment_sum / bs1.tweet_count) - (bs2.sentiment_sum / bs2.tweet_count) AS Average_sent_diff
            FROM `binned_sentiment` bs1, `binned_sentiment` bs2
            WHERE bs1.cID = bs2.cID AND bs2.bin_position = bs1.bin_position + 1 """
cursor.execute(query)
sentiment_change = cursor.fetchall()

In [ ]:
df_sentiment_change = pd.DataFrame(sentiment_change, dtype = float)
df_sentiment_change.rename(columns ={0:"Airline",1:"Average Sentiment Change"}, inplace=True)
df_sentiment_change

In [ ]:
sns.boxplot(data=df_sentiment_change,x='Airline',y='Average Sentiment Change', order= ['British Airways','Other','AmericanAir'
                                                                                       ]).set_title('Average Sentiment Change per Airline')